# Import Statements

In [50]:
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package punkt to /Users/kimberly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import OCR_pipeline as OCR
import TopicModeling as TM
import jsonpickle

# Cleaning And Saving Extracted Text

In [52]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="XXX",
  database="forestry"
)
mycursor = mydb.cursor()

def getExtractedTexts():
    mycursor.execute("SELECT filename, extractedText FROM PolicyPapers")
    myresult = mycursor.fetchall()
    return [x for x in myresult]

def getExtractedTextsFromCountries(countries):
    sqlStatement = "SELECT filename, extractedText FROM PolicyPapers WHERE"
    
    for i in range(len(countries)):
        whereClause = " country = '%s' " % countries[i]
        sqlStatement += whereClause
        if len(countries) != 1 and i < len(countries) - 1:
            sqlStatement += "or"
            
    mycursor.execute(sqlStatement)
    myresult = mycursor.fetchall()
    return [x for x in myresult]

In [53]:
key_words = ['afforestation','agriculture','animal welfare','artificial regeneration','biodiversity',
             'biological resources','biome','board','clean','coconut','conservation','control','database',
             'enforcement','environment','farm','financing','forest','forest protection','funding','fundraising',
             'land ','land use','landholder','law','measuring','mobilization','monitor','natural resources',
             'oversight','plant breeders','plants','pollution','preservation','produce','protection','qualification',
             'quality','registry','regulation','reporting','reserve','resource','restriction','results-driven',
             'rural','safeguard','species' ,'support','sustainable','technical submission','threatened species',
             'variety','verification','wastewater','water','watershed','wild life']

In [54]:
spanish_key_words = ['repoblación forestal', 'agricultura', 'bienestar de los animales', 'regeneración artificial', 
                     'biodiversidad', 'recursos biologicos', 'bioma', 'tablero', 'limpiar', 'Coco', 'conservación',
                     'controlar', 'base de datos', 'aplicación', 'ambiente', 'granja', 'financiación', 'bosque', 
                     'protección forestal', 'fondos', 'recaudación de fondos', 'tierra', 'uso del suelo', 
                     'terrateniente', 'ley', 'medición', 'movilización', 'monitor', 'recursos naturales', 'vigilancia',
                     'fitomejoradores', 'plantas', 'contaminación', 'preservación', 'Produce', 'proteccion',
                     'calificación', 'calidad', 'registro', 'regulación', 'reportando', 'reserva', 'recurso', 
                     'restricción', 'impulsado por resultados', 'rural', 'salvaguardia', 'especies', 'apoyo', 
                     'sostenible', 'sumisión técnica', 'especies amenazadas', 'variedad', 'verificación', 
                     'aguas residuales', 'agua', 'cuenca', 'fauna silvestre']

In [68]:
#dictionary for texts in terms of sentences
import re
import gc
textDictionary = {}
countries = ['India']
policyPapers = getExtractedTextsFromCountries(countries)
language_keywords = {}
language_keywords['english'] = key_words
language_keywords['spanish'] = spanish_key_words
for paper in policyPapers:
    cleanWhitespaces = re.sub(r'(.) ', r'\1', paper[1])
    cleanedText = TM.toSentences(cleanWhitespaces, 'english', language_keywords['english'])
    textDictionary[paper[0]] = cleanedText

# LDA

In [69]:
from pprint import pprint
lemmatized_sents = []

# Print the Keywords in the 5 topics
for key in textDictionary:
    cleanedText = textDictionary[key]
    lemmatized_sents.extend(cleanedText)
    
corpus, lda_model = TM.LDA(lemmatized_sents)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
#Next Step: Store LDA data in Policy Paper Object & other central file

[(28,
  '0.001*"source" + 0.001*"commission" + 0.001*"reduce" + 0.001*"tail" + '
  '0.001*"toxic" + 0.001*"useable" + 0.001*"abate" + 0.001*"arise" + '
  '0.001*"noise" + 0.001*"minimum"'),
 (13,
  '0.001*"source" + 0.001*"commission" + 0.001*"reduce" + 0.001*"tail" + '
  '0.001*"toxic" + 0.001*"useable" + 0.001*"abate" + 0.001*"arise" + '
  '0.001*"noise" + 0.001*"minimum"'),
 (10,
  '0.001*"mean" + 0.001*"mine" + 0.001*"thereof" + 0.001*"mining" + '
  '0.001*"act" + 0.001*"bureau" + 0.001*"plan" + 0.001*"rule" + '
  '0.001*"controller" + 0.001*"indian"'),
 (22,
  '0.001*"area" + 0.001*"mining" + 0.001*"lease" + 0.001*"surface" + 0.001*"_" '
  '+ 0.001*"boundary" + 0.001*"metre" + 0.001*"follow" + 0.001*"iii" + '
  '0.001*"hectare"'),
 (1,
  '0.038*"unproductive" + 0.038*"unplanned" + 0.038*"certain" + '
  '0.038*"drilling" + 0.002*"mean" + 0.001*"mine" + 0.001*"bureau" + '
  '0.001*"thereof" + 0.001*"indian" + 0.001*"agent"'),
 (8,
  '0.029*"ground" + 0.029*"mineral" + 0.029*"undergr

In [41]:
my = lda_model.print_topics()[0][1]

In [40]:
re.findall(r"[a-zA-Z]+",my)

['capacity',
 'adaptive',
 'percentage',
 'planning',
 'gomp',
 'state',
 'atla',
 'number',
 'area',
 'university']

## Extract sentences by Keywords

### TODO: Have a feature on the web app so that the user can extract text using keywords

In [ ]:
#Try with the first document
policy_0 = policyPapers[0][1]
sentences = TM.ReturnSentence(policy_0)

In [73]:
#This can be used as a preview feature
for s in sentences:
    if 'land' in s:
        print(s)

 (2) Notwithstanding anything contained in sub-rule (1) a scheme of prospecting, shall include: (a) baseline information of prevailing environmental conditions before the beginning of prospecting operations; (b) [steps proposed to be taken for protection of environment which will include] prevention and control of air and water pollution, progressive reclamation and rehabilitation of the land disturbed by the prospecting operations, a scheme for the plantation of trees, and such other measures, as may be directed from time to time by the Controller General or the authorised officer for minimising the adverse effect of prospecting operations on the environment
 Types of plans and sections: (1) The owner, agent, mining engineer or manager of every mine shall keep the following plans and sections(a) a surface plan showing every surface feature within the mining lease boundaries, such as building, telephone, telegraph or power transmission line, watermain, tramline, railway, road, river, w

## Use Key Words to classify documents

In [48]:
key_words = ['afforestation','agriculture','animal welfare','artificial regeneration','biodiversity',
             'biological resources','biome','board','clean','coconut','conservation','control','database',
             'enforcement','environment','farm','financing','forest','forest protection','funding','fundraising',
             'land ','land use','landholder','law','measuring','mobilization','monitor','natural resources',
             'oversight','plant breeders','plants','pollution','preservation','produce','protection','qualification',
             'quality','registry','regulation','reporting','reserve','resource','restriction','results-driven',
             'rural','safeguard','species' ,'support','sustainable','technical submission','threatened species',
             'variety','verification','wastewater','water','watershed','wild life']

In [58]:
'artificial regeneration' in cleanWhitespaces

False

In [68]:
# Create a list with each entry being whether or not containing the keyword
if_key_word = [int(word in cleanWhitespaces) for i, word in enumerate(key_words)]

In [69]:
if_key_word

[0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0]

# Visualizations

### pyLDAvis

`pip install pyldavis`

Code source: https://markroxor.github.io/gensim/static/notebooks/gensim_news_classification.html
<br>
The area of the circles represent the prevalance of the topic. The length of the bars on the right represent the membership of a term in a particular topic. 

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)

### WordCloud

`pip install wordcloud`

Code source: https://medium.com/@rudyb2001/making-sense-of-the-news-using-machine-learning-e13ed5f96cf0
<br>
WordCloud of Top 10 words in each topic with the sizes of the words proportional to the weights.

In [ ]:
len(lda_model.show_topics(formatted=False))

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from wordcloud import WordCloud, STOPWORDS
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 5, figsize=(15,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

# Storage